In [1]:
import pandas as pd
import numpy as np
from keras.layers import Activation, Dense, Dropout, SpatialDropout1D,Input,Masking,Bidirectional, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from random import seed
#seed(1)
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
import math
from sklearn.metrics import roc_auc_score
import itertools
from sklearn.metrics import classification_report, confusion_matrix
from prettytable import PrettyTable

#defining the early stopping criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,restore_best_weights=True,patience=3)

Using TensorFlow backend.


In [ ]:
gs = pd.read_csv('lstm_analysis.csv')

In [ ]:
gs

In [48]:
gs.drop('Unnamed: 0',axis=1,inplace=True)
gs.drop('Unnamed: 0.1',axis=1,inplace=True)

In [49]:
def split_70(x):
    return int((round((x/15)*0.7))*15)

In [50]:
cols_to_use = ['uhid','pulserate', 'dischargestatus']

In [27]:
def range_finder(p):
    length = p
    fractional = (p/15.0) - math.floor(p/15.0)
    return int(round(fractional*15))

def randomize(u):
    
    f_df = pd.DataFrame(columns=u.columns)
    for i in u.uhid.unique():
        print(i)
        x = u[u['uhid']==i]
        x = x[range_finder(len(x)):len(x)]

        f_df = f_df.append(x,ignore_index=True)
        
    return f_df
    
def make_lstm(gd):


    final_df = randomize(gd)
    


    final_df.fillna(-999,inplace=True)


    # In[ ]:

    death_cases = final_df[final_df.dischargestatus == 1]
    discharge_cases = final_df[final_df.dischargestatus == 0]
    
    train_death = death_cases[:split_70(len(death_cases))]
    test_death = death_cases[split_70(len(death_cases)):]
    
    train_discharge = discharge_cases[:split_70(len(discharge_cases))]
    test_discharge = discharge_cases[split_70(len(discharge_cases)):]
    
    train = pd.concat([train_death,train_discharge])
    test = pd.concat([test_death,test_discharge])



    # In[ ]:


    y_train = train['dischargestatus']
    X_train = train.drop('dischargestatus',axis=1)
    X_train = X_train.drop('uhid',axis=1)
    #X_train = X_train.drop('visittime',axis=1)

    y_test = test['dischargestatus']
    X_test = test.drop('dischargestatus',axis=1)
    X_test = X_test.drop('uhid',axis=1)
    #X_test = X_test.drop('startdate',axis=1)


    # In[ ]:

    auc_roc_inter = []
    val_a = []
    train_a = []


    #converting the data into a numpy array
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    ytrain1 = []
    for i in range(0,len(y_train),15):
        #print(i)
        y1 = y_train[i:i+15]
        ytrain1.append(y1[-1])

    ytest1 = []
    for i in range(0,len(y_test),15):
        #print(i)
        y1 = y_test[i:i+15]
        ytest1.append(y1[-1])

    ytrain1 = np.array(ytrain1)
    ytest1 = np.array(ytest1)

    Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
    Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

    return Xtrain,Xtest,ytrain1,ytest1

def lstm_model(Xtrain,Xtest,ytrain1,ytest1):
    auc_roc_inter = []
    val_a = []
    train_a = []
    for i in range(2):
        #Building the LSTM model
        X = Input(shape=(None, 1), name='X')
        mX = Masking()(X)
        lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
        mX = lstm(mX)
        L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
        y = Dense(1, activation="sigmoid")(L)
        outputs = [y]
        inputs = [X]
        model = Model(inputs,outputs)
        model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

        v_a = []
        t_a = []
        #fitting the model
        model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        #history = model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
        for i in range(len(model.history.history['val_accuracy'])):
            v_a.append(model.history.history['val_accuracy'][i])
            t_a.append(model.history.history['accuracy'][i])
        #predictions
        y_pred = model.predict(Xtest)
        #y_pred = y_pred.round()
        y_test = np.array(ytest1)
        y_pred = np.array(y_pred)
        y_test = pd.DataFrame(y_test)
        y_test = np.array(y_test)

        def acc(x):
            if x>0.5:
                return 1
            else:
                return 0

        y_model=[]
        for i in y_pred:
            y_model.append(acc(i))
        y_answer=[]
        for j in y_test:
            y_answer.append(acc(j))
            
        val_a.append(v_a)
        train_a.append(t_a)
        auc_roc_inter.append(roc_auc_score(y_answer,y_pred))
        continue
    
    
        
    return auc_roc_inter,y_model,y_answer



In [8]:
gd = gs[cols_to_use]

In [16]:
len(gs)

978682

In [24]:
gd.dischargestatus.unique()

array([0, 1])

In [72]:
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)

RSHI.0000023452
RNEH.0000012581
RSHI.0000020326
RSHI.0000014720
RSHI.0000012985
RSHI.0000015691
RSHI.0000023451
RSHI.0000015178
RNEH.0000008375
RNEH.0000011301
RSHI.0000012088
RNEH.0000013016
RSHI.0000017471
RSHI.0000022896
RSHI.0000017472
RNEH.0000013396
RSHI.0000019707
RSHI.0000021953
RNEH.0000009255
RSHI.0000015211
RNEH.0000013713
RSHI.0000016373
                   uhid  pulserate  dischargestatus
0       RSHI.0000023452      138.0                0
1       RSHI.0000023452     -999.0                0
2       RSHI.0000023452      138.0                0
3       RSHI.0000023452      140.0                0
4       RSHI.0000023452      140.0                0
5       RSHI.0000023452      139.0                0
6       RSHI.0000023452      139.0                0
7       RSHI.0000023452      139.0                0
8       RSHI.0000023452      138.0                0
9       RSHI.0000023452      138.0                0
10      RSHI.0000023452      137.0                0
11      RSHI.0000023452 

In [78]:
#print(ytrain1)
#[x for x in ytrain1 if x == 1]
#[x for x in ytrain1 if x == 0]
#[x for x in ytest1 if x == 1]
[x for x in ytest1 if x == 0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)

Train on 38816 samples, validate on 6850 samples
Epoch 1/38
27600/38816 [====================>.........] - ETA: 2:25 - loss: 0.6933 - accuracy: 0.5245

In [42]:
death = gs[gs['dischargestatus']==1]
dis = gs[gs['dischargestatus']==0]
auc_pulserate = an[0]
pulserate_dea = (float(len(death[death['pulserate']==-999]))/len(death))*100
pulserate_dis = (float(len(dis[dis['pulserate']==-999]))/len(death))*100

In [44]:
cols_to_use = ['uhid','heartrate', 'dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)
auc_heartrate = an[0]
heartrate_dea = (float(len(death[death['heartrate']==-999]))/len(death))*100
heartrate_dis = (float(len(death[death['heartrate']==-999]))/len(death))*100

Train on 29202 samples, validate on 5154 samples
Epoch 1/38
29202/29202 [==============================] - 50s 2ms/step - loss: 0.6542 - accuracy: 0.5949 - val_loss: 0.9105 - val_accuracy: 0.0392
Epoch 2/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6510 - accuracy: 0.5985 - val_loss: 0.9097 - val_accuracy: 0.0390
Epoch 3/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6514 - accuracy: 0.5971 - val_loss: 0.9135 - val_accuracy: 0.0392
Epoch 4/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6517 - accuracy: 0.5961 - val_loss: 0.8484 - val_accuracy: 0.0390
Epoch 5/38
29202/29202 [==============================] - 48s 2ms/step - loss: 0.6505 - accuracy: 0.5981 - val_loss: 0.9023 - val_accuracy: 0.0394
Epoch 6/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6506 - accuracy: 0.5983 - val_loss: 0.9720 - val_accuracy: 0.0388
Epoch 7/38
29202/29202 [==============================] - 48s 2ms/ste

In [45]:
cols_to_use = ['uhid','ecg_resprate', 'dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)
auc_rr = an[0]
rr_dea = (float(len(death[death['ecg_resprate']==-999]))/len(death))*100
rr_dis = (float(len(death[death['ecg_resprate']==-999]))/len(death))*100

Train on 29202 samples, validate on 5154 samples
Epoch 1/38
29202/29202 [==============================] - 50s 2ms/step - loss: 0.3072 - accuracy: 0.8762 - val_loss: 0.2258 - val_accuracy: 0.8853
Epoch 2/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.3020 - accuracy: 0.8811 - val_loss: 0.2494 - val_accuracy: 0.8851
Epoch 3/38
29202/29202 [==============================] - 48s 2ms/step - loss: 0.3033 - accuracy: 0.8793 - val_loss: 0.2337 - val_accuracy: 0.8855
Epoch 4/38
29202/29202 [==============================] - 48s 2ms/step - loss: 0.3046 - accuracy: 0.8795 - val_loss: 0.2322 - val_accuracy: 0.8855
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping
Train on 29202 samples, validate on 5154 samples
Epoch 1/38
29202/29202 [==============================] - 50s 2ms/step - loss: 0.3060 - accuracy: 0.8799 - val_loss: 0.2350 - val_accuracy: 0.8853
Epoch 2/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.300

In [46]:
cols_to_use = ['uhid','spo2', 'dischargestatus']
gd = gs[cols_to_use]
Xtrain,Xtest,ytrain1,ytest1 = make_lstm(gd)
an,y_model,y_answer = lstm_model(Xtrain,Xtest,ytrain1,ytest1)
auc_spo2 = an[0]
spo2_dea = (float(len(death[death['spo2']==-999]))/len(death))*100
spo2_dis = (float(len(death[death['spo2']==-999]))/len(death))*100

Train on 29202 samples, validate on 5154 samples
Epoch 1/38
29202/29202 [==============================] - 51s 2ms/step - loss: 0.6723 - accuracy: 0.5833 - val_loss: 0.1531 - val_accuracy: 0.9775
Epoch 2/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6678 - accuracy: 0.5919 - val_loss: 0.1710 - val_accuracy: 0.9777
Epoch 3/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6637 - accuracy: 0.5979 - val_loss: 0.1147 - val_accuracy: 0.9777
Epoch 4/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6639 - accuracy: 0.5969 - val_loss: 0.1644 - val_accuracy: 0.9777
Epoch 5/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6627 - accuracy: 0.5991 - val_loss: 0.1127 - val_accuracy: 0.9777
Epoch 6/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6612 - accuracy: 0.6024 - val_loss: 0.1720 - val_accuracy: 0.9777
Epoch 7/38
29202/29202 [==============================] - 49s 2ms/ste

In [47]:
l = [["Pulserate" ,auc_pulserate, pulserate_dea,pulserate_dea],["Heartrate" ,auc_heartrate, heartrate_dea,heartrate_dea],["RR" ,auc_rr, rr_dea,rr_dea],["SpO2" ,auc_spo2, spo2_dea,spo2_dea]]

table = PrettyTable(['Parameter', 'AUC-ROC', '% Death','% Discharge'])

for rec in l:
    table.add_row(rec)

print(table)

+-----------+--------------------+---------------+---------------+
| Parameter |      AUC-ROC       |    % Death    |  % Discharge  |
+-----------+--------------------+---------------+---------------+
| Pulserate | 0.5313389938375837 | 2.41327914383 | 2.41327914383 |
| Heartrate | 0.697626929953715  |  12.15734504  |  12.15734504  |
|     RR    | 0.8264447110902694 | 21.9261165726 | 21.9261165726 |
|    SpO2   | 0.3928552050809734 | 2.27427600268 | 2.27427600268 |
+-----------+--------------------+---------------+---------------+
